In [17]:
import os
import sys

sys.path.insert(0, '/kaggle/input/rag-utils')

from subprocess_runner import run_command
from file_operations import parse_file, save_embeddings, load_embeddings
from embedding_operations import get_chunks_embeddings, get_prompt_embedding, find_most_similar, query_model_with_context

os.chdir('/kaggle/working/')
print(os.getcwd())

/kaggle/working


In [7]:
run_command("curl -fsSL https://ollama.com/install.sh | sh")
os.system("/usr/local/bin/ollama serve &")

/usr/lib/python3.10/subprocess.py:961: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stdout = io.open(c2pread, 'rb', bufsize)
/usr/lib/python3.10/subprocess.py:966: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stderr = io.open(errread, 'rb', bufsize)


0

In [8]:
run_command("ollama pull mistral")
run_command("curl http://127.0.0.1:11434/api/chat -d '{\"model\": \"mistral\", \"stream\": false, \"messages\": [{ \"role\": \"user\", \"content\": \"Write a sentence as J. D. Salinger would write it\" }]}'")

'{"model":"mistral","created_at":"2025-01-20T14:28:33.561080829Z","message":{"role":"assistant","content":" \\"On a whimsy of a Tuesday morning, as the sun danced across the windowpane, Holden Caulfield found himself musing over a seemingly ordinary moment, yet discovering in it the extraordinary revelation of his own existential confusion.\\""},"done_reason":"stop","done":true,"total_duration":3627057020,"load_duration":2245170239,"prompt_eval_count":18,"prompt_eval_duration":351000000,"eval_count":54,"eval_duration":1029000000}'

In [10]:
filename = "../input/peter-pan/peter_pan.txt.txt"
paragraphs = parse_file(filename)

In [11]:
embeddings = get_chunks_embeddings(filename, "mistral", paragraphs)
print("Embeddings generated successfully!")

Processing chunks: 100%|██████████| 1736/1736 [09:55<00:00,  2.92chunk/s]


Embeddings generated successfully!


In [14]:
SYSTEM_PROMPT = """You are a helpful reading assistant who answers questions 
    based on snippets of text provided in context. Answer only using the context provided, 
    being as concise as possible. If you're unsure, just say that you don't know.
    Context:
"""

prompt = "Who's the story primary villain?"

prompt_embedding = get_prompt_embedding(prompt, modelname="mistral")
most_similar_chunks = find_most_similar(prompt_embedding, embeddings)[:5]

In [18]:
response = query_model_with_context(
    modelname="mistral",
    system_prompt=SYSTEM_PROMPT,
    prompt=prompt,
    most_similar_chunks=most_similar_chunks,
    paragraphs=paragraphs,
)

if response is not None:
    print("Model response:")
    print(response['content'])
else:
    print("Failed to get a response from the model.")

Model response:
 The context does not provide enough information to determine who the primary villain in the story is.
